<a href="https://colab.research.google.com/github/Krishna-Kumar-Sankaran-Kutty/mfCompare/blob/main/GetMFData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyze MF Portfolio data from Money Control

In [ ]:
#Import Dependencies
import gradio as gr
from bs4 import BeautifulSoup
import requests
import pandas as pd
import plotly.express as px

In [ ]:
# Define function to extract portfolio details from given url from Moneycontrol Portfolio Holdings page
def extract_portfolio(url):
  # Send an HTTP GET request to the URL
  response = requests.get(url)

  # Parse the HTML content using BeautifulSoup
  soup = BeautifulSoup(response.content, 'html.parser')

  # Find the table element
  table = soup.find('table', id='equityCompleteHoldingTable')

  # Extract table headers
  headers = [header.text.strip() for header in table.find_all('th')]

  # Extract table rows
  rows = []
  for row in table.find_all('tr')[1:]:  # Skip the header row
    data = [cell.text.strip() for cell in row.find_all('td')]
    rows.append(data)

  # Create a Pandas DataFrame
  df = pd.DataFrame(rows, columns=headers)

  # Add Fund Name column
  df['Fund Name'] = url.split('/')[-3].replace('-', ' ').title()

  # Set Fund Name column as index
  #df = df.set_index('Fund Name')

  # Clean-up 'Stock Invested in' column
  df['Stock Invested in'] = df['Stock Invested in'].str.replace('#\n', '')

  # Clean-up '% of Total Holdings' column nad convert to float
  df['% of Total Holdings'] = df['% of Total Holdings'].str.rstrip('%').astype(float)

  # Drop all columns except 'Stock Invested in', 'Sector Total', and '% of Total Holdings'
  df = df[['Fund Name', 'Sector', 'Stock Invested in', '% of Total Holdings']]

  return df

In [ ]:
# Define function to generate treemap based on selected funds
def generate_treemap(fund1, fund2, fund3):
  fund_urls = {
      "Hdfc Index Fund Nifty 50 Plan Direct Plan": 'https://www.moneycontrol.com/mutual-funds/hdfc-index-fund-nifty-50-plan-direct-plan/portfolio-holdings/MHD1152',
      "Parag Parikh Flexi Cap Fund Direct Plan": 'https://www.moneycontrol.com/mutual-funds/parag-parikh-flexi-cap-fund-direct-plan/portfolio-holdings/MPP002',
      "Hdfc Nifty Next 50 Index Fund Direct Plan": 'https://www.moneycontrol.com/mutual-funds/hdfc-nifty-next-50-index-fund-direct-plan/portfolio-holdings/MHD3484'
  }

  selected_funds = [fund1, fund2, fund3]
  dfs = []
  for fund in selected_funds:
    if fund:
      url = fund_urls[fund]
      df = extract_portfolio(url)
      dfs.append(df)

  if dfs:
    combined_df = pd.concat(dfs)
    fig = px.treemap(combined_df,
                    path=[px.Constant("All Funds"), 'Fund Name', 'Sector', 'Stock Invested in'],
                    values='% of Total Holdings',
                    color='Fund Name',
                    template='ggplot2')
    return fig
  else:
    return None

In [ ]:
# Create Gradio interface
fund_choices = [
    "Hdfc Index Fund Nifty 50 Plan Direct Plan",
    "Parag Parikh Flexi Cap Fund Direct Plan",
    "Hdfc Nifty Next 50 Index Fund Direct Plan"
]

iface = gr.Interface(
    fn=generate_treemap,
    inputs=[
        gr.Markdown("# MF Portfolio Analysis #"),
        gr.Markdown("## Created by KK ##"),
        gr.Dropdown(choices=fund_choices, label="Fund 1"),
        gr.Dropdown(choices=fund_choices, label="Fund 2"),
        gr.Dropdown(choices=fund_choices, label="Fund 3")
],
    outputs=gr.Plot()
)

iface.launch()

/usr/local/lib/python3.10/dist-packages/gradio/utils.py:1002: UserWarning:

Expected 3 arguments for function <function generate_treemap at 0x7f422665c790>, received 5.

/usr/local/lib/python3.10/dist-packages/gradio/utils.py:1010: UserWarning:

Expected maximum 3 arguments for function <function generate_treemap at 0x7f422665c790>, received 5.



Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c33815bd363689d435.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
